In [77]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector

from mmcv import Config
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import test
from mmdet.apis import train_detector, init_detector, inference_detector
import torch 



In [60]:
!wget https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_320_273e_coco/yolov3_d53_320_273e_coco-421362b6.pth

--2021-10-28 11:25:50--  https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_320_273e_coco/yolov3_d53_320_273e_coco-421362b6.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248091419 (237M) [application/octet-stream]
Saving to: ‘yolov3_d53_320_273e_coco-421362b6.pth.2’

yolov3_d53_320_273e 100%[===================>] 236.60M   429KB/s    in 8m 44s  

2021-10-28 11:34:35 (462 KB/s) - ‘yolov3_d53_320_273e_coco-421362b6.pth.2’ saved [248091419/248091419]



In [78]:
cfg = Config.fromfile('./configs/yolo/yolov3_d53_320_273e_coco.py')
cfg.dataset_type = 'CocoDataset'
#print(cfg.classes)
#cfg.classes = ("person",)
# number of classes
#cfg.model.bbox_head.num_classes = 1

In [79]:
PREFIX = '/home/franz/RTTS/rtts_coco/'
# cfg.data.train.img_prefix = PREFIX
# cfg.data.train.classes = cfg.classes
# cfg.data.train.ann_file = PREFIX + 'train_annotations.json'
# cfg.data.train.type = 'CocoDataset'
# cfg.data.val.img_prefix = PREFIX
# cfg.data.val.classes = cfg.classes
# cfg.data.val.ann_file = PREFIX + 'val_annotations.json'
# cfg.data.val.type = 'CocoDataset'

cfg.data.test.img_prefix = '/home/franz/RTTS/rtts_coco/'
#cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file = PREFIX + '_test.json'
cfg.data.test.type = 'CocoDataset'

In [80]:
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.policy = 'step'
cfg.lr_config.warmup = 'linear'
cfg.log_config.interval = 600
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'bbox'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

#cfg.load_from = None
#cfg.work_dir = "../hazyir"
cfg.runner.max_epochs = 12
cfg.total_epochs = 12

In [81]:
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.test)]

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


In [65]:
dir(test)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'collect_results_cpu',
 'collect_results_gpu',
 'dist',
 'encode_mask_results',
 'get_dist_info',
 'mmcv',
 'multi_gpu_test',
 'osp',
 'pickle',
 'shutil',
 'single_gpu_test',
 'tempfile',
 'tensor2imgs',
 'time',
 'torch']

In [66]:
from box import Box

In [67]:
args = Box({'checkpoint' : 'yolov3_d53_320_273e_coco-421362b6.pth',
            'work-dir' : '',
            'out' : 'results.pkl',
            'fuse-conv-bn' : '',
            'format-only' : '',
            'eval' : 'mAP',
            'show' : False,
            'show-dir' : False,
            'show-score-thr' : 0.3,
            'gpu-collect': False,
            'tmpdir' : True,
            'cfg-options' : '',
            'options' : '',
            'eval-options' : '',
            'launcher' : 'none'})

In [69]:
#args = parse_args()

assert args.out or args.eval or args.format_only or args.show \
    or args.show_dir, \
    ('Please specify at least one operation (save/eval/format/show the '
     'results / save the results) with the argument "--out", "--eval"'
     ', "--format-only", "--show" or "--show-dir"')

if args.eval and args.format_only:
    raise ValueError('--eval and --format_only cannot be both specified')

if args.out is not None and not args.out.endswith(('.pkl', '.pickle')):
    raise ValueError('The output file must be a pkl file.')

# cfg = Config.fromfile(args.config)
# if args.cfg_options is not None:
#     cfg.merge_from_dict(args.cfg_options)
# # import modules from string list.
# if cfg.get('custom_imports', None):
#     from mmcv.utils import import_modules_from_strings
#     import_modules_from_strings(**cfg['custom_imports'])
# # set cudnn_benchmark
# if cfg.get('cudnn_benchmark', False):
#     torch.backends.cudnn.benchmark = True

# cfg.model.pretrained = None
# if cfg.model.get('neck'):
#     if isinstance(cfg.model.neck, list):
#         for neck_cfg in cfg.model.neck:
#             if neck_cfg.get('rfp_backbone'):
#                 if neck_cfg.rfp_backbone.get('pretrained'):
#                     neck_cfg.rfp_backbone.pretrained = None
#     elif cfg.model.neck.get('rfp_backbone'):
#         if cfg.model.neck.rfp_backbone.get('pretrained'):
#             cfg.model.neck.rfp_backbone.pretrained = None

# in case the test dataset is concatenated
samples_per_gpu = 1
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    samples_per_gpu = cfg.data.test.pop('samples_per_gpu', 1)
    if samples_per_gpu > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    samples_per_gpu = max(
        [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in cfg.data.test])
    if samples_per_gpu > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)

rank, _ = get_dist_info()
# allows not to create
if args.work_dir is not None and rank == 0:
    mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=False)

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model)
# model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))


fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, args.checkpoint, map_location='cpu')
if args.fuse_conv_bn:
    model = fuse_conv_bn(model)
# old versions did not save class info in checkpoints, this walkaround is
# for backward compatibility
if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

model.CLASSES = dataset.CLASSES

    
if not distributed:
    model = MMDataParallel(model, device_ids=[0])
    outputs = single_gpu_test(model, data_loader, args.show, args.show_dir,
                              args.show_score_thr)
else:
    model = MMDistributedDataParallel(
        model.cuda(),
        device_ids=[torch.cuda.current_device()],
        broadcast_buffers=False)
    outputs = multi_gpu_test(model, data_loader, args.tmpdir,
                             args.gpu_collect)

rank = 0
if rank == 0:
    if args.out:
        print(f'\nwriting results to {args.out}')
        mmcv.dump(outputs, args.out)
    kwargs = {} if args.eval_options is None else args.eval_options
    if args.format_only:
        dataset.format_results(outputs, **kwargs)
    if args.eval:
        eval_kwargs = cfg.get('evaluation', {}).copy()
        # hard-code way to remove EvalHook args
        for key in [
                'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
                'rule'
        ]:
            eval_kwargs.pop(key, None)
        #eval_kwargs.update(dict(metric=args.eval, **kwargs))
        metric = dataset.evaluate(outputs)
        print(metric)
        metric_dict = dict(config=args.config, metric=metric)
        if args.work_dir is not None and rank == 0:
            mmcv.dump(metric_dict, json_file)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4322/4322, 31.2 task/s, elapsed: 138s, ETA:     0s
writing results to results.pkl


IndexError: list index out of range

In [71]:
outputs[0]

[array([[ 4.18473236e+02,  1.23647881e+02,  4.64488556e+02,
          2.61188843e+02,  9.58725035e-01],
        [ 2.87346130e+02,  9.92197876e+01,  3.03484833e+02,
          1.18402756e+02,  3.37982401e-02],
        [ 4.00641541e+02,  2.13302979e+02,  4.23901154e+02,
          2.70708893e+02,  3.04881055e-02],
        [ 4.66854309e+02,  1.41801270e+02,  4.78639282e+02,
          2.03603683e+02,  1.25896689e-02],
        [ 4.67545654e+02,  1.63131256e+02,  4.79557648e+02,
          2.30432816e+02,  8.98510963e-03],
        [ 4.04684052e+02,  1.41345673e+02,  4.33374268e+02,
          2.57343292e+02,  7.82588497e-03],
        [ 4.65197693e+02,  1.77172012e+02,  4.77811676e+02,
          2.61947479e+02,  5.69706736e-03],
        [ 1.31495422e+02,  1.67131607e+02,  3.79451813e+02,
          2.64603149e+02,  4.45614289e-03],
        [ 2.69178891e+00,  1.03845619e+02,  3.14317970e+01,
          2.13900970e+02,  3.25425877e-03],
        [-4.28529453e+00, -5.56276178e+00,  1.75410294e+02,
    

In [95]:
metric = dataset.evaluate(outputs, metric='bbox')


Evaluating bbox...
Loading and preparing results...
DONE (t=0.53s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=11.72s).
Accumulating evaluation results...
DONE (t=2.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.189
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.145
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.141
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.141
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.141
 Average Recall     (AR) @[ IoU=0.50:0.95

In [96]:
metric

OrderedDict([('bbox_mAP', 0.093),
             ('bbox_mAP_50', 0.189),
             ('bbox_mAP_75', 0.085),
             ('bbox_mAP_s', 0.014),
             ('bbox_mAP_m', 0.082),
             ('bbox_mAP_l', 0.145),
             ('bbox_mAP_copypaste', '0.093 0.189 0.085 0.014 0.082 0.145')])

In [91]:
dataset.CLASSES

('person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush')

In [ ]:
list()

In [94]:
dataset.cat_ids = list(range(len(dataset.CLASSES)))